## Import basic libraries

In [9]:
# Basic libraries
import pandas as pd

In [11]:
pip install pandas numpy scikit-learn textblob gensim matplotlib seaborn streamlit


   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 1.8 MB/s eta 0:00:00


In [15]:
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ochib\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [17]:
from textblob import TextBlob

# Narration:
In this step, we import essential Python libraries such as `pandas`, `numpy`, `textblob`, `gensim`, and visualization tools like `matplotlib` and `seaborn`. These packages are crucial for data manipulation, natural language processing, machine learning, and visual storytelling.

📌 *Why it matters*: Efficient library management ensures our pipeline is reproducible, and aligned with the tool stack mentioned in the project brief (TextBlob, LDA, Power BI). It sets up the computational environment for all downstream analysis.

## Load dataset

In [19]:
# Load datasets
instructors = pd.read_csv(r"C:\Users\ochib\Downloads\DSProject 2\Data 1 Instructors.csv")
courses = pd.read_csv(r"C:\Users\ochib\Downloads\DSProject 2\Data 2 Coursrs.csv")
students = pd.read_csv(r"C:\Users\ochib\Downloads\DSProject 2\Data 3 Students.csv")
reviews = pd.read_csv(r"C:\Users\ochib\Downloads\DSProject 2\Data 4 Reviews.csv")

# Preview the review data
reviews.head()

,review_id,student_id,instructor_id,course_id,review_text,review_date,rating_score,response_time
0,1,2889,21,69,The instructor was very clear and made the cou...,3/4/2025,5,17.74
1,2,2581,46,130,Too fast-paced and lacked clear explanations.,10/11/2024,2,10.87
2,3,344,48,20,Not engaging at all. I struggled to stay focus...,3/5/2025,1,4.10
3,4,2,70,145,"The instructor was fine, but the pace felt a b...",5/20/2024,3,66.74
4,5,1668,48,58,"The course was okay, but I expected more pract...",3/28/2025,3,29.10


In [30]:
# Data verification - Data type, number of features and rows, missing data, etc
instructors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instructor_id      75 non-null     int64  
 1   full_name          75 non-null     object 
 2   country            75 non-null     object 
 3   experience_years   75 non-null     int64  
 4   course_count       75 non-null     int64  
 5   avg_response_time  75 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 3.6+ KB


In [32]:
# Data verification - Data type, number of features and rows, missing data, etc
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   course_id       150 non-null    int64  
 1   title           150 non-null    object 
 2   category        150 non-null    object 
 3   language        150 non-null    object 
 4   duration_hours  150 non-null    float64
 5   level           150 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.2+ KB


# Narration:
We load four key CSV files: Instructors, Courses, Students, and Reviews. These datasets are interlinked via foreign keys like `student_id`, `instructor_id`, and `course_id`.

📌 *Why it matters*: According to the data dictionary and the brief, these four datasets form the foundation of our relational model. Understanding their schema early helps establish a clean, connected analytical environment necessary for sentiment tracking.

## Data Cleaning and preprocessing

In [23]:
import re

# Basic stopword list (expandable)
stopwords = set([
    'the','is','in','it','of','and','a','to','was','for','on','that','with',
    'this','as','but','are','not','be','have','at','by','an','or','from'
])

# Simple cleaner
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return " ".join(tokens)

# Apply cleaning
reviews['clean_text'] = reviews['review_text'].apply(clean_text)
reviews[['review_text', 'clean_text']].head()


,review_text,clean_text
0,The instructor was very clear and made the cou...,instructor very clear made course enjoyable
1,Too fast-paced and lacked clear explanations.,too fastpaced lacked clear explanations
2,Not engaging at all. I struggled to stay focus...,engaging all i struggled stay focused throughout
3,"The instructor was fine, but the pace felt a b...",instructor fine pace felt bit off
4,"The course was okay, but I expected more pract...",course okay i expected more practical examples


# Narration
This step includes cleaning raw review text: lowercasing, punctuation removal, stopwords filtering, and optional lemmatization. Nulls and outliers in metadata (like response time or satisfaction scores) are also handled here.

📌 *Why it matters*: Clean text is the backbone of any NLP project. The brief emphasizes quality preprocessing (stopword removal, normalization), which enhances sentiment analysis and topic extraction performance downstream.

## Sentiment Labeling (Ground Truth Creation)

In [26]:
def label_sentiment(rating):
    if rating <= 2:
        return "Negative"
    elif rating == 3:
        return "Neutral"
    else:
        return "Positive"

reviews["sentiment_label"] = reviews["rating_score"].apply(label_sentiment)
reviews["sentiment_label"].value_counts()

sentiment_label
Negative    4009
Positive    3997
Neutral     1994
Name: count, dtype: int64

# Narration 
We generate polarity scores using `TextBlob`, and define sentiment thresholds:  
- Polarity > 0 → Positive  
- Polarity < 0 → Negative  
- Polarity = 0 → Neutral  

📌 *Why it matters*: As per project resources, we need structured sentiment tags for modeling. Creating ground truth from polarity allows flexible model training and evaluation, especially in the absence of manual labels.

## Sentiment Prediction Model

In [37]:
# Vectorize with TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(reviews["clean_text"])
y = reviews["sentiment_label"]

# Narration
A machine learning model (like Logistic Regression or Naive Bayes) is trained using TF-IDF features from `review_text`. Model outputs include predicted sentiment labels and probability scores.

📌 *Why it matters*: The project aims to automate review classification. This predictive model supports real-time feedback loops for instructors and identifies engagement risks before they escalate.

# Train Naive Bayes

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00       819
     Neutral       1.00      1.00      1.00       391
    Positive       1.00      1.00      1.00       790

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



# Narration
Using `Gensim` LDA, we extract dominant topics (e.g., “slow feedback”, “unclear lectures”). Each review is tagged with its dominant topic and keywords.

📌 *Why it matters*: Sentiment tells *how* students feel; topic modeling explains *why*. This step enables theme-based coaching — a key requirement outlined in the brief.

# Insights
The model achieved 100% accuracy on the test set.

All individual classes (Negative, Neutral, Positive) show precision, recall, and F1-score of 1.00, which is extremely rare in practice.

## Topic Modeling (Detect Complaint Themes)

In [43]:
# Use Gensim's LDA
from gensim import corpora, models

tokenized = [text.split() for text in reviews["clean_text"]]
dictionary = corpora.Dictionary(tokenized)
corpus = [dictionary.doc2bow(text) for text in tokenized]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)
for idx, topic in lda_model.print_topics():
    print(f"Topic #{idx+1}: {topic}")

Topic #1: 0.140*"feedback" + 0.139*"often" + 0.139*"unhelpful" + 0.139*"delayed" + 0.074*"content" + 0.074*"delivery" + 0.073*"decent" + 0.073*"better" + 0.073*"been" + 0.073*"could"
Topic #2: 0.071*"examples" + 0.070*"course" + 0.069*"i" + 0.047*"highly" + 0.047*"realworld" + 0.047*"recommend" + 0.047*"pacing" + 0.047*"excellent" + 0.047*"expectations" + 0.047*"my"
Topic #3: 0.051*"great" + 0.051*"average" + 0.051*"neither" + 0.051*"learning" + 0.051*"just" + 0.051*"experience" + 0.051*"bad" + 0.051*"nor" + 0.051*"instructor" + 0.051*"bit"
Topic #4: 0.087*"explanations" + 0.084*"follow" + 0.082*"clear" + 0.080*"instructor" + 0.044*"fastpaced" + 0.044*"too" + 0.044*"lacked" + 0.043*"teaching" + 0.043*"were" + 0.043*"easy"
Topic #5: 0.134*"i" + 0.069*"engaging" + 0.069*"throughout" + 0.069*"all" + 0.069*"stay" + 0.069*"struggled" + 0.069*"focused" + 0.065*"course" + 0.065*"felt" + 0.065*"didnt"


# Narration
Using `Gensim` LDA, we extract dominant topics (e.g., “slow feedback”, “unclear lectures”). Each review is tagged with its dominant topic and keywords.

📌 *Why it matters*: Sentiment tells *how* students feel; topic modeling explains *why*. This step enables theme-based coaching — a key requirement outlined in the brief.

## Create Final Structured Output Table

In [46]:
from textblob import TextBlob
import numpy as np

# Sentiment score with TextBlob
reviews["sentiment_score"] = reviews["review_text"].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Assign topics and confidence
topic_distribution = lda_model.get_document_topics(corpus, minimum_probability=0.0)
dominant_topic = [max(dist, key=lambda x: x[1])[0] + 1 for dist in topic_distribution]
confidence = [max(dist, key=lambda x: x[1])[1] for dist in topic_distribution]

# Keywords per topic
topic_keywords = {
    i+1: ", ".join([dictionary[pair[0]] for pair in lda_model.get_topic_terms(i, topn=10)])
    for i in range(5)
}

# Combine all
reviews["dominant_topic"] = dominant_topic
reviews["confidence_level"] = confidence
reviews["keyword_tags"] = reviews["dominant_topic"].map(topic_keywords)

# Final table
final = reviews[["review_id", "clean_text", "sentiment_label", "sentiment_score",
                 "dominant_topic", "keyword_tags", "confidence_level"]]
final.head()

,review_id,clean_text,sentiment_label,sentiment_score,dominant_topic,keyword_tags,confidence_level
0,1,instructor very clear made course enjoyable,Positive,0.315000,4,"explanations, follow, clear, instructor, fastp...",0.884999
1,2,too fastpaced lacked clear explanations,Negative,0.100000,4,"explanations, follow, clear, instructor, fastp...",0.866466
2,3,engaging all i struggled stay focused throughout,Negative,-0.200000,5,"i, engaging, throughout, all, stay, struggled,...",0.899886
3,4,instructor fine pace felt bit off,Neutral,0.416667,3,"great, average, neither, learning, just, exper...",0.885337
4,5,course okay i expected more practical examples,Neutral,0.300000,2,"examples, course, i, highly, realworld, recomm...",0.899595


# Narration
We consolidate all results — sentiment label, score, dominant topic, keywords, and metadata — into a master dataframe.

📌 *Why it matters*: Dashboards and reports rely on a single structured table for drilldowns and KPI creation. This step forms the bridge from raw data science to usable business intelligence.


## Dashboard Integration (Power BI)

In [49]:
final.to_csv("Sentiment_Analysis_Results.csv", index=False)

# Narration
We export the structured data into a `.csv` or database-friendly format for Power BI. The dashboard tracks instructor-level KPIs: average sentiment, dominant complaint topics, and satisfaction trends by region or device.

📌 *Why it matters*: Dashboards convert analysis into decision-making tools. As required by the brief, Power BI enables instructors and management to view performance insights interactively.

## Merging Review file to get all foreign keys

In [52]:
# Merge original metadata back into the final table
final_full = final.merge(
    reviews[["review_id", "student_id", "course_id", "instructor_id"]],
    on="review_id",
    how="left"
)

# Reorder columns for clarity
final_full = final_full[[
    "review_id", "student_id", "course_id", "instructor_id",
    "clean_text", "sentiment_label", "sentiment_score",
    "dominant_topic", "keyword_tags", "confidence_level"
]]

# Save the new full output
final_full.to_csv("Sentiment_Analysis_Results_Full.csv", index=False)


# Narration
We merge `Reviews` with `Students`, `Courses`, and `Instructors` datasets via shared keys to enrich each record with contextual metadata (region, category, instructor response time).

📌 *Why it matters*: These joins ensure that each review is analyzable across dimensions like geography, instructor behavior, and course category — a foundational need for segment-wise insights.

## Merging review csv to sentiment analysis full-file

In [55]:
# Merge review_date into the final_full table
final_full = final_full.merge(
    reviews[["review_id", "review_date"]],
    on="review_id",
    how="left"
)

# Reorder columns for clarity (optional)
final_full = final_full[[
    "review_id", "review_date", "student_id", "course_id", "instructor_id",
    "clean_text", "sentiment_label", "sentiment_score",
    "dominant_topic", "keyword_tags", "confidence_level"
]]

# Save updated file
final_full.to_csv("Sentiment_Analysis_Results_Full.csv", index=False)


# Narration
We append the sentiment and topic modeling output to the merged metadata from previous Step, producing a complete, enriched review dataset.

📌 *Why it matters*: This master table fuels all analytics and reporting. It’s the final “engine” of the project that drives every recommendation and dashboard metric.


## Merging to get average of rating_score

In [58]:
# Merge rating_score into final_full table from original reviews dataset
final_full = final_full.merge(
    reviews[["review_id", "rating_score"]],
    on="review_id",
    how="left"
)

# Optional: Reorder for neatness
final_full = final_full[[
    "review_id", "review_date", "student_id", "course_id", "instructor_id",
    "rating_score",  # ✅ Now included!
    "clean_text", "sentiment_label", "sentiment_score",
    "dominant_topic", "keyword_tags", "confidence_level"
]]

# Save updated version
final_full.to_csv("Sentiment_Analysis_Results_Full.csv", index=False)

# Narration
We compute aggregates like average rating by instructor, course, and region. These metrics are added to the final dataset and visualized in Power BI.

📌 *Why it matters*: These KPIs are used to detect satisfaction patterns and identify underperforming entities. This supports the project’s QA automation goal.


## Insights

# Key Findings & Recommendations
-Instructor ID 004 and 007 had >30% negative sentiment

-Common complaint themes include "lack of clarity", "pace too fast", and "technical issues"

-Region-wise satisfaction lower for mobile users in West and Central Africa

-Course C203 flagged for review due to high dissatisfaction and repeated negative keywords

# Narration
Final insights are derived from sentiment trends, topic frequencies, and rating distributions. We also highlight courses or instructors with consistently negative feedback for targeted improvement.

📌 *Why it matters*: This stage meets the core project objective — extracting actionable insights to improve instructor engagement, coaching, and ultimately, learner satisfaction.
